# BM25

Notebooks to show our BM25 method and its performances.

## Import

In [1]:
from bm25 import BM25
import pandas as pd
import data_loader
from metrics import export_query_result_to_submission_csv, get_answers_for_submission_csv, compute_recall_10


%load_ext autoreload
%autoreload 2

## BM25 Initialization

In [2]:
bm25 = BM25("../data/corpus.json/clean_corpus_fr.json", "../data/train.csv", k1=1, b=0.9)

Loading corpus...
Corpus loaded successfully !

Information about the given corpus
###################################
Number of documents: 10676
Language (only one language should be displayed): ['fr']

Loading queries...
Queries loaded successfully !

Information about the given queries
###################################
Number of queries: 1608
Language (only one language should be displayed): ['fr']



Computing IDF: 100%|██████████| 705413/705413 [00:00<00:00, 1173661.67it/s]


## Document Retrival

Retrieve top10 doc for every query

In [3]:
top10_docid_per_query = bm25.top_10_docid_for_all_queries()

Ranking queries: 100%|██████████| 1608/1608 [01:45<00:00, 15.19it/s]


In [4]:
queries_id = list(top10_docid_per_query.keys())
docids = list(top10_docid_per_query.values())
export_query_result_to_submission_csv(queries_id, docids)

Compute recall@10

In [5]:
get_answers_for_submission_csv()
compute_recall_10()

0.29850746268656714